In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import math

In [ ]:
train_data = pd.read_csv("/kaggle/input/ml-for-oceanography/train.csv")
test_data = pd.read_csv("/kaggle/input/ml-for-oceanography/test1.csv")
sample_data = pd.read_csv("/kaggle/input/ml-for-oceanography/sample2.csv")

In [ ]:
train_data.head(5)

In [ ]:
X_train = train_data
X_test =  test_data
# X_train = train_data.drop(["ID",],axis=1)
# X_test = test_data.drop(["ID",],axis=1)

In [ ]:
X_train["wavelength"] = (9.8*(X_train["DPD(sec)"])**2)

In [ ]:
X_train["mean_wavelength"] =  9.8*(X_train["APD(sec)"])**2

In [ ]:
X_test["wavelength"] = (9.8*(X_test["DPD(sec)"])**2)
X_test["mean_wavelength"] =  9.8*(X_test["APD(sec)"])**2

In [ ]:
X_train.corr()

In [ ]:
X_train1 = X_train.drop('WVHT(m)',axis=1)
y_train1 = X_train['WVHT(m)']

In [ ]:
import xgboost
import sklearn
xgb_regressor = xgboost.XGBRegressor()

In [ ]:
## Hyper Parameter Optimization
booster=['gbtree','gblinear']
base_score=[0.25,0.5,0.75]
n_estimators = [100, 500, 900, 1100, 1500]
max_depth = [2, 3, 5]
learning_rate=[0.1,0.01,0.05,0.001]
min_child_weight=[2,3,4,6,8]
gamma = [0,0.01,0.001,0.1,1,2,5]
colsample_bytree = [0.1,0.3,0.4,0.5]
subsample = [0.1,0.2,0.3,0.4,0.5]

# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth':max_depth,
    'learning_rate':learning_rate,
    'min_child_weight':min_child_weight,
    'booster':booster,
    'base_score':base_score,
    'gamma':gamma,
    'colsample_bytree':colsample_bytree,
    'subsample':subsample
    }
# Set up the random search with 5-fold cross validation
random_cv = sklearn.model_selection.RandomizedSearchCV(estimator=xgb_regressor,
            param_distributions=hyperparameter_grid,
            cv=5, n_iter=50,
            scoring = 'neg_mean_squared_error',n_jobs = 4,
            verbose = 3, 
            return_train_score = True,
            random_state=42)

In [ ]:
# random_cv.fit(X_train1,y_train1)

In [ ]:
# print(random_cv.best_estimator_.base_score,
#       random_cv.best_estimator_.booster,
#       random_cv.best_estimator_.learning_rate,
#       random_cv.best_estimator_.max_depth,
#       random_cv.best_estimator_.min_child_weight, 
#       random_cv.best_estimator_.n_estimators,
#       random_cv.best_estimator_.gamma,
#       random_cv.best_estimator_.colsample_bytree,
#       random_cv.best_estimator_.subsample)
#0.5 gbtree 0.05 2 2 1100 0.1 0.4 0.4

In [ ]:
xgb1=xgboost.XGBRegressor(base_score=0.5, booster="gbtree",
                          learning_rate=0.05,max_depth=2,
                          min_child_weight=2, 
                          n_estimators=1100,gamma=0.1,colsample_bytree=0.4,subsample=0.4,random_state=15)

xgb1.fit(X_train1,y_train1)


In [ ]:
from sklearn.metrics import mean_squared_error



In [ ]:
print(mean_squared_error(xgb1.predict(X_train1), y_train1,squared=False))

In [ ]:
booster=['gbtree']
base_score=[0.5]

n_estimators = [1100,2000]
max_depth = [2, 5]
learning_rate=[0.05]
min_child_weight=[2,5]
gamma = [0.1]
colsample_bytree = [0.4,0.5]
subsample = [0.4,0.5]

# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth':max_depth,
    'learning_rate':learning_rate,
    'min_child_weight':min_child_weight,
    'booster':booster,
    'base_score':base_score,
    'gamma':gamma,
    'colsample_bytree': colsample_bytree,
    "subsample":subsample
    }
grid_cv = sklearn.model_selection.GridSearchCV(estimator=xgb_regressor,
            param_grid=hyperparameter_grid,
            cv=5,
            scoring = 'neg_mean_absolute_error',n_jobs = 1,
            verbose = 3, 
            return_train_score = True)

In [ ]:
# grid_cv.fit(X_train1,y_train1)

In [ ]:
# print(grid_cv.best_estimator_.base_score,
#       grid_cv.best_estimator_.booster,
#       grid_cv.best_estimator_.learning_rate,
#       grid_cv.best_estimator_.max_depth,
#       grid_cv.best_estimator_.min_child_weight, 
#       grid_cv.best_estimator_.n_estimators,
#       grid_cv.best_estimator_.gamma,
#       grid_cv.best_estimator_.colsample_bytree,
#       grid_cv.best_estimator_.subsample)
# #0.5 gbtree 0.05 5 5 1100 0.1 0.5 0.5

In [ ]:
xgb2=xgboost.XGBRegressor(base_score=0.5, booster="gbtree",
                          learning_rate=0.05,max_depth=5,
                          min_child_weight=5 ,
                          n_estimators=1100,gamma=0.1,colsample_bytree=0.5,subsample=0.5,random_state=15)
xgb2.fit(X_train1,y_train1)

In [ ]:
print(mean_squared_error(xgb2.predict(X_train1), y_train1,squared=False))

In [ ]:
print(mean_squared_error((xgb1.predict(X_train1)+xgb2.predict(X_train1))/2, y_train1,squared=False))

> Catboosting


In [ ]:
import catboost

In [ ]:
cb_regressor = catboost.CatBoostRegressor(silent=True)

In [ ]:
## Hyper Parameter Optimization
learning_rate=[0.01,0.008,0.005,0.001]
depth=[4,6,8,10]
l2_leaf_reg = [2,4,6,8,10]
random_strength =  [0,2,4,6,8,10]

# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'learning_rate':learning_rate,
    'depth':depth,
    'l2_leaf_reg':l2_leaf_reg,
    'random_strength':random_strength
    }
# Set up the random search with 5-fold cross validation
random_cv = sklearn.model_selection.RandomizedSearchCV(estimator=cb_regressor,
            param_distributions=hyperparameter_grid,
            cv=5, n_iter=50,
            scoring = 'neg_mean_squared_error',n_jobs = 4,
            verbose = 3, 
            return_train_score = True,
            random_state=42)

In [ ]:
# random_cv.fit(X_train1,y_train1)

In [ ]:
# print(random_cv.best_estimator_.get_param("learning_rate"),
#      random_cv.best_estimator_.get_param("depth"),
#      random_cv.best_estimator_.get_param("l2_leaf_reg"),
#      random_cv.best_estimator_.get_param("random_strength")
#      )
# 0.01 4 2 0

In [ ]:
cb1=catboost.CatBoostRegressor(learning_rate=0.01,depth=4,
                          l2_leaf_reg=2, 
                          random_strength=0,silent=True)

cb1.fit(X_train1,y_train1)

In [ ]:
print(mean_squared_error(cb1.predict(X_train1), y_train1,squared=False))

Gradient Boosting

In [ ]:
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor

In [ ]:
gbr = GradientBoostingRegressor(verbose=3)

In [ ]:
## Hyper Parameter Optimization
learning_rate=[0.1,0.05,0.01]
n_estimators =[500,1000,2000,3000]
min_samples_split = [2,3,4,5 ]
max_depth=[3,4,5,6,7,8]

# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'learning_rate':learning_rate,
    'max_depth':max_depth,
    'min_samples_split':min_samples_split,
    'n_estimators':n_estimators
    }
# Set up the random search with 5-fold cross validation
random_cv = sklearn.model_selection.RandomizedSearchCV(estimator=gbr,
            param_distributions=hyperparameter_grid,
            cv=5, n_iter=10,
            scoring = 'neg_mean_squared_error',n_jobs = 4,
            verbose = 3, 
            return_train_score = True,
            random_state=42)

In [ ]:
# random_cv.fit(X_train1,y_train1)

In [ ]:
# print(random_cv.best_estimator_.min_samples_split,
#       random_cv.best_estimator_.learning_rate,
#       random_cv.best_estimator_.max_depth,
#       random_cv.best_estimator_.n_estimators,)
# 2 0.01 7 500

In [ ]:
gbr = GradientBoostingRegressor(verbose=0)

learning_rate=[0.01]
n_estimators =[500]
min_samples_split = [10,20,50,60]
max_depth=[7,8]
max_features = [2,3,4]
subsample = [0.2,0.5,0.6]



# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'learning_rate':learning_rate,
    'max_depth':max_depth,
    'min_samples_split':min_samples_split,
    'n_estimators':n_estimators,
    'max_features':max_features,
    "subsample":subsample
    }
grid_cv = sklearn.model_selection.GridSearchCV(estimator=gbr,
            param_grid=hyperparameter_grid,
            cv=5,
            scoring = 'neg_mean_absolute_error',n_jobs = 1,
            verbose = 3, 
            return_train_score = True)

In [ ]:
# grid_cv.fit(X_train1,y_train1)

In [ ]:
# print(grid_cv.best_estimator_.min_samples_split,
#      grid_cv.best_estimator_.max_features,
#      grid_cv.best_estimator_.subsample)
# 20 4 0.2

In [ ]:
gbr1 = GradientBoostingRegressor(n_estimators =500 , max_depth=7,learning_rate=0.01,min_samples_split=20,max_features=4,subsample=0.2,random_state=15)
gbr1.fit(X_train1,y_train1)

In [ ]:
print(mean_squared_error(gbr1.predict(X_train1), y_train1,squared=False))

Stacking Models

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [ ]:
from sklearn.linear_model import Lasso, Ridge
import lightgbm as lgb

#elnet = ElasticNet(alpha=0.01,l1_ratio=0.05)
gbr3 = GradientBoostingRegressor(n_estimators =500 , max_depth=7,learning_rate=0.01,
                                 min_samples_split=20,max_features=4,subsample=0.2,random_state=15)
#svr1 = SVR(kernel="rbf",gamma=0.001,C=10) 
lgbm1 = lgb.LGBMRegressor(boosting_type="gbdt",learning_rate=0.1,max_depth=10,max_bin=12,
                          num_leaves=4,colsample_bytree=0.5,subsample=0.4,lambda_l2=10)
xgb1=xgboost.XGBRegressor(base_score=0.5, booster="gbtree",
                          learning_rate=0.05,max_depth=2,
                          min_child_weight=2, 
                          n_estimators=1100,gamma=0.1,colsample_bytree=0.4,subsample=0.4,random_state=15)

# xgb2=xgboost.XGBRegressor(base_score=0.5, booster="gbtree",
#                           learning_rate=0.08,max_depth=5,
#                           min_child_weight=5 ,
#                           n_estimators=500,gamma=0.05,colsample_bytree=0.5,subsample=0.4)

lasso = Ridge(alpha =10, random_state=1)
stacked_averaged_models = StackingAveragedModels(base_models = (gbr3,xgb1,lgbm1),
                                                 meta_model = lasso)

In [ ]:
from sklearn.model_selection import KFold, cross_val_score, train_test_split
n_folds = 5
kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train1)


rmse = np.sqrt(-cross_val_score(stacked_averaged_models, np.array(X_train1), np.array(y_train1), scoring="neg_mean_squared_error", cv = kf))

In [ ]:
print(rmse.mean())

In [ ]:
stacked_averaged_models.fit(np.array(X_train1),np.array(y_train1))
print(mean_squared_error(stacked_averaged_models.predict(np.array(X_train1)),np.array(y_train1),squared=False))

Lightgbm

In [ ]:
import lightgbm as lgb
# lgbm1 = lgb.LGBMRegressor(objective='regression',num_leaves=5,
#                               learning_rate=0.05, n_estimators=720,
#                               max_bin = 55, bagging_fraction = 0.8,
#                               bagging_freq = 5, feature_fraction = 0.2319,
#                               feature_fraction_seed=9, bagging_seed=9,
#                               min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)


#lgbm1 = lgb.LGBMRegressor(boosting_type="gbdt",colsample_bytree=0.3,subsample=0.5,reg_lambda=10)
lgbm  = lgb.LGBMRegressor()

In [ ]:
## Hyper Parameter Optimization
num_leaves = [2,4,6,8] 
max_depth = [2,4,8,16,32,64]
subsample = [0.2,0.3,0.4,0.5,0.6]
colsample_bytree = [0.3,0.4,0.5,0.6]
max_bin = [2,4,6,8,10]
lambda_l2 = [0.1,1,2,10,100]
learning_rate = [0.1,0.01,0.001]



# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'learning_rate':learning_rate,
    'max_depth':max_depth,
    'num_leaves':num_leaves,
    'subsample':subsample,
    'colsample_bytree':colsample_bytree,
    'max_bin':max_bin,
    'lambda_l2':lambda_l2}
# Set up the random search with 5-fold cross validation
random_cv = sklearn.model_selection.RandomizedSearchCV(estimator=lgbm,
            param_distributions = hyperparameter_grid,
            cv=5, n_iter=50,
            scoring = 'neg_mean_squared_error',n_jobs = 4,
            verbose = 3, 
            return_train_score = True,
            random_state=42)

# lgbm1.fit(np.array(X_train1),np.array(y_train1))
# print(mean_squared_error(lgbm1.predict(np.array(X_train1)),np.array(y_train1),squared=False))

In [ ]:
# # Random CV 
# random_cv.fit(X_train1,y_train1)

In [ ]:
# print(random_cv.best_estimator_.colsample_bytree,
#       random_cv.best_estimator_.lambda_l2,
#       random_cv.best_estimator_.learning_rate,
#       random_cv.best_estimator_.max_depth,
#       random_cv.best_estimator_.max_bin, 
#       random_cv.best_estimator_.num_leaves,
#       random_cv.best_estimator_.subsample)
# # 0.5 10 0.1 2 10 8 0.4


In [ ]:
lgbm1 = lgb.LGBMRegressor(boosting_type="gbdt",learning_rate=0.1,max_depth=2,max_bin=10,
                          num_leaves=8,colsample_bytree=0.5,subsample=0.4,lambda_l2=10)
lgbm1.fit(np.array(X_train1),np.array(y_train1))
print(mean_squared_error(lgbm1.predict(np.array(X_train1)),np.array(y_train1),squared=False))

GridSearchCV

In [ ]:
## Hyper Parameter Optimization
num_leaves = [4,8,12] 
max_depth = [2,10]
subsample = [0.4,0.6]
colsample_bytree = [0.3,0.5]
max_bin = [10,12]
lambda_l2 = [10]
learning_rate = [0.1,0.05]



# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'learning_rate':learning_rate,
    'max_depth':max_depth,
    'num_leaves':num_leaves,
    'subsample':subsample,
    'colsample_bytree':colsample_bytree,
    'max_bin':max_bin,
    'lambda_l2':lambda_l2}
# Set up the random search with 5-fold cross validation
grid_cv = sklearn.model_selection.GridSearchCV(estimator=lgbm,
            param_grid = hyperparameter_grid,
            cv=5,
            scoring = 'neg_mean_squared_error',n_jobs = 4,
            verbose = 3, 
            return_train_score = True)


In [ ]:
# grid_cv.fit(X_train1,y_train1)

In [ ]:
# print(grid_cv.best_estimator_.colsample_bytree,
#       grid_cv.best_estimator_.lambda_l2,
#       grid_cv.best_estimator_.learning_rate,
#       grid_cv.best_estimator_.max_depth,
#       grid_cv.best_estimator_.max_bin, 
#       grid_cv.best_estimator_.num_leaves,
#       grid_cv.best_estimator_.subsample,
#      grid_cv.best_estimator_.boosting_type)
# 0.5 10 0.1 10 12 4 0.4 gbdt

In [ ]:
lgbm1 = lgb.LGBMRegressor(boosting_type="gbdt",learning_rate=0.1,max_depth=10,max_bin=12,
                          num_leaves=4,colsample_bytree=0.5,subsample=0.4,lambda_l2=10)
lgbm1.fit(np.array(X_train1),np.array(y_train1))
print(mean_squared_error(lgbm1.predict(np.array(X_train1)),np.array(y_train1),squared=False))

In [ ]:
print(mean_squared_error(( 0.15*xgb2.predict(X_train1))+( 0.15*cb1.predict(X_train1)) +(0.7*stacked_averaged_models.predict(np.array(X_train1))),
                         y_train1,squared=False))


In [ ]:
sample_data["WVHT(m)"] = 0.7*(stacked_averaged_models.predict(np.array(X_test))) + (0.15*cb1.predict(X_test) ) + (0.15*xgb2.predict(X_test))

In [ ]:
sample_data.to_csv("submssion_file.csv",index=False)

In [ ]:
sample_data